In [1]:
# Import các module cần thiết
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [2]:
# Khởi tạo một phiên Spark
spark = SparkSession.builder \
    .appName("LinearRegressionExample") \
    .getOrCreate()

24/03/29 15:53:21 WARN Utils: Your hostname, codespaces-f38966 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/03/29 15:53:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/29 15:53:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Tạo dữ liệu giả định và chuyển đổi thành DataFrame PySpark
data = [(0.0, 1.0), (1.0, 2.0), (2.0, 3.0), (3.0, 4.0)]
df = spark.createDataFrame(data, ["label", "features"])
df.show()

+-----+--------+
|label|features|
+-----+--------+
|  0.0|     1.0|
|  1.0|     2.0|
|  2.0|     3.0|
|  3.0|     4.0|
+-----+--------+



In [7]:
# Tạo một đối tượng VectorAssembler để tạo vectơ đặc trưng từ cột "features"
assembler = VectorAssembler(
    inputCols=["features"],
    outputCol="features_vector"
)

In [8]:

# Chuyển đổi DataFrame để thêm cột features_vector
df = assembler.transform(df)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_data, test_data = df.randomSplit([0.8, 0.2], seed=1234)

In [9]:
# Khởi tạo mô hình Linear Regression
lr = LinearRegression(featuresCol="features_vector", labelCol="label")

In [10]:
# Huấn luyện mô hình trên tập huấn luyện
lr_model = lr.fit(train_data)

24/03/29 15:54:30 WARN Instrumentation: [c2d0ea4e] regParam is zero, which might cause numerical instability and overfitting.
24/03/29 15:54:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/03/29 15:54:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/03/29 15:54:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [11]:
# Dự đoán trên tập kiểm tra
predictions = lr_model.transform(test_data)

In [12]:
# Đánh giá mô hình bằng cách tính MSE trên tập kiểm tra
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mse")
mse = evaluator.evaluate(predictions)
print("Mean Squared Error trên tập kiểm tra:", mse)

Mean Squared Error trên tập kiểm tra: 7.888609052210118e-31


In [13]:
# Hiển thị các hệ số của mô hình
print("Hệ số của mô hình:", lr_model.coefficients)

Hệ số của mô hình: [0.9999999999999987]


In [14]:
# Đóng phiên Spark
spark.stop()
